In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets
from IPython.display import display

# Function to generate histogram based on k, Eb/N0, and d inputs
def generate_histogram(k, EbN0_db, d):
    M = 60000  # Number of symbols
    nsamp = 16  # Number of samples per symbol

    L = 2 ** k  # Number of levels
    SNR_db = EbN0_db - 10 * np.log10(nsamp / (2 * k))  # Calculate SNR in dB
    SNR = 10 ** (SNR_db * 0.1)  # Convert SNR from dB to linear scale

    # Generate random signal with L levels
    x = (2 * np.floor(L * np.random.rand(M)) - L + 1) * d / 2

    # Power of the original signal
    Px = (d ** 2 / 4) * (L ** 2 - 1) / 3
    Measured_x = np.sum(x ** 2) / len(x)

    # Upsample the signal
    y = np.repeat(x, nsamp)

    # Generate Gaussian noise
    noise = np.random.normal(0, np.sqrt(Measured_x / SNR), len(y))
    y_noisy = y + noise

    # Matched filter (average over nsamp samples)
    y_reshaped = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp, 1))
    z = np.matmul(y_reshaped, matched) / nsamp

    # Set up the bins for the histogram
    A = np.arange(-(L - 1) * d / 2, L * d / 2, d)

    # Plot the histogram
    fig, ax = plt.subplots(1, 1, figsize=(14, 4))
    ax.hist(z, bins=200, edgecolor='white', color='#1F77B4')
    ax.set_xticks(A)
    ax.set_xlabel("Integers")
    ax.set_ylabel("Frequency")
    ax.legend([f"Eb/N0 = {EbN0_db} dB"])
    ax.set_title('Histogram of the Noisy Signal')

    plt.show()

# UI Components for input
k_input = widgets.IntText(value=3, description='k:', continuous_update=False)
d_input = widgets.IntText(value=5, description='d:', continuous_update=False)
EbN0_db_slider = widgets.FloatSlider(value=12, min=0, max=20, step=0.1, description='Eb/N0 (dB):', continuous_update=False)

# Arrange inputs in a VBox
inputs = widgets.VBox([k_input, d_input, EbN0_db_slider])

# Interactive output to update the plot based on k, d, and Eb/N0 values
out = widgets.interactive_output(generate_histogram, {'k': k_input, 'd': d_input, 'EbN0_db': EbN0_db_slider})

# Display the UI and output
display(inputs, out)


Output()